# 15. Attribute Selection Measures

Decision trees recursively split features with regard to their target label's purity. Attribute selection measures are designed to find the **optimal point** of the **most predictive feature** in order to split 1 dataset into 2. These 2 new datasets’ target variable will be more pure than the original dataset’s. Purity means how homogeneous a dataset is.

How do we measure the **purity** of the 2 new datasets?
The two most popular algorithms for measuring purity and selecting the optimal decisions in a decision tree are **Information Entropy** and **Gini Index**.

We will discuss how these two methods measure purty using three examples:

- If we have 4 red gumballs and 0 blue gumballs, that group of 4 is 100% pure.
- If we have 2 red and 2 blue, that group is 100% impure.
- If we have 3 red and 1 blue, that group is either 75% or 81% pure, if we use Gini or Entropy respectively.

<img src="https://github.com/rhennig/EMA6938/blob/main/Notebooks/Figures/Impurity.png?raw=1" alt="Impurity Examples" align="center" style="width:400px; float:center"/>